### Import Packages

In [188]:
import numpy as np
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import folium
from folium import plugins

from pandas.io.json import json_normalize
%matplotlib inline

### Load 1 Year of Data

In [2]:
# startDate = "2017-01-01"
# datelist = pd.date_range(pd.to_datetime(startDate), periods=366).tolist() #1 year of data

In [3]:
# requestsDataFrame = pd.DataFrame()

# for index in range(len(datelist)-1):
#     day = str(datelist[index].date())
#     nextDay = str(datelist[index+1].date())
#     url = """https://secure.toronto.ca/webwizard/ws/requests.json?start_date=""" + day + """T00:00:00Z&end_date=""" + nextDay + """T00:00:00Z&jurisdiction_id=toronto.ca"""
#     response = requests.get(url)
#     response = json_normalize(response.json()['service_requests'])
#     assert response.shape[0] < 1000
#     requestsDataFrame = pd.concat([requestsDataFrame, response])

In [38]:
# requestsDataFrame.to_pickle("./rawData.pkl")

In [197]:
requestsDataFrame = pd.read_pickle("rawData.pkl")

### Basic data cleaning

In [198]:
requestsDataFrame.reset_index(inplace = True)
requestsDataFrame.drop(['index'], axis = 1, inplace = True)

In [199]:
requestsDataFrame.requested_datetime = pd.to_datetime(requestsDataFrame.requested_datetime).dt.date
requestsDataFrame.expected_datetime = pd.to_datetime(requestsDataFrame.expected_datetime).dt.date
requestsDataFrame.updated_datetime = pd.to_datetime(requestsDataFrame.updated_datetime).dt.date

In [200]:
#remove duplicate service_request_id's -> take latest updated datetime
rows = requestsDataFrame.groupby("service_request_id")['updated_datetime'].transform(max) == requestsDataFrame['updated_datetime']
requestsDataFrame = requestsDataFrame[rows]

requestsDataFrame = requestsDataFrame.drop_duplicates()

### Assertions

In [201]:
#check if no duplicate service_request_id's
assert requestsDataFrame.shape[0] == len(requestsDataFrame['service_request_id'].unique())

In [202]:
# Ensure number of days == 365
assert len(pd.to_datetime(requestsDataFrame.requested_datetime).dt.date.unique()) == 365

### EDA

In [203]:
columns = ["service_request_id", "lat", "long", "service_name", "requested_datetime", "expected_datetime",
           "updated_datetime", "status", "status_notes"]
requestsDataFrame = requestsDataFrame[columns]

In [220]:
pd.DataFrame(requestsDataFrame.groupby('service_name')['service_name'].count())

,service_name
service_name,
Bridge - Graffiti Complaint,363
Graffiti,2127
Litter / Bin / Graffiti on Bin,22
Road - Graffiti Complaint,111
Road - Pot hole,11511
Sidewalk - Graffiti Complaint,284


In [213]:
pd.DataFrame(requestsDataFrame['status'].unique())

,0
0,closed
1,open


In [212]:
pd.DataFrame(requestsDataFrame['status_notes'].unique())

,0
0,Completed - The request has been concluded.
1,In progress - The request is being investigate...
2,"Cancelled - The request may be a duplicate, wo..."
3,In progress - The request has been scheduled.


In [171]:
requestsDataFrame['long']

0       -79.404203
1       -79.400065
2       -79.386904
3       -79.286311
4       -79.385589
5       -79.398370
6       -79.352485
7       -79.494080
8       -79.265247
9       -79.300042
10      -79.251502
11      -79.434046
12      -79.352485
13      -79.434654
14      -79.406815
15      -79.408519
16      -79.339652
17      -79.373258
18      -79.406532
19      -79.200578
20      -79.464518
21      -79.337722
22      -79.390558
23      -79.399765
24      -79.496774
25      -79.324818
26      -79.502016
27      -79.358419
28      -79.434009
29      -79.408576
           ...    
14388   -79.408519
14389   -79.270524
14390   -79.475066
14391   -79.351002
14392   -79.359828
14393   -79.501506
14394   -79.475886
14395   -79.403766
14396   -79.490618
14397   -79.536267
14398   -79.414481
14399   -79.420087
14400   -79.408519
14401   -79.400285
14402   -79.490367
14403   -79.271431
14404   -79.489841
14405   -79.352485
14406   -79.292636
14407   -79.287205
14408   -79.542384
14409   -79.

In [52]:
requestsDataFrame['service_name'].unique()

array(['Graffiti', 'Road - Pot hole', 'Road - Graffiti Complaint',
       'Sidewalk - Graffiti Complaint', 'Bridge - Graffiti Complaint',
       'Litter / Bin / Graffiti on Bin'], dtype=object)